# Statistics and Violin Plotting to Excel

## What this notebook does:

This notebook loops through all "#_ Condition_DDMMYY.csv" data files in the working directory and calculates ROI count, mean, median, and standard deviation. The statistics cell also runs Shapiro-Wilk tests of normal distribution and creates Histograms and Probability Plots for each file, then saves them to a  _Histograms_ProbPlots_ folder in the working directory. For each input file, the statistics cell writes the appropriate animal number, hours post denervation, and innervation status in separate columns and outputs a new .csv to the _Appended_CSVs_ folder that is created in the working directory. The statistics cell writes all values, histograms, and probability plots to respective  **Statistics_DDMMYY.xlsx** _Statistics_ or _Histograms_ProbPlots_ worksheets. Subsequent cells calculate innervated:denervated mean intensity ratio and run Mann-Whitney U tests between innervated and denervated data by animal, then writes these to _Statistics_. Later optional cells create Plotly violin plots and Seaborn violin plots to their respective _Plotly_PNGs_, _Plotly_HTMLs_, and _Seaborn_Violins_ folders then export these to the respective **Statistics_DDMMYY.xlsx** _Comparison_Violins_ and _Individual_Violins_ worksheets. The **Statistics_DDMMYY.xlsx** outfile then saves and closes.

## Using this notebook:

1. Check that this notebook is within the folder that contains .csv data files.

2. Check file names are correct: #_Condition_DDMMYY.csv, where # is the animal number, Condition is case-sensitive and one of (Innervated, Denervated, Uninjured, Control), and DDMMYY is the date .csvs were created.

3. Change the DDMMYY in the first cell to align with file names. It doesn't have to be specifically "DDMMYY" formatted, as long as all files have the same ending.

4. Add or remove animals from animals list or hours from hours post denervation (hpd) list in first cell as needed. Edit hpd_sort( ) and violin_df_sort(df) accordingly (see below).

5. Removable cells are indicated; these cells can be deleted with no effect on other cells.

6. When a cell finishes its function it will print "Done". This doesn't mean everything ran perfectly, only that no error was raised.

7. The circle in the top left under "Logout" will be colored in while the code is running, and empty when it is not. Once the circle is empty after hitting "run", scroll to the bottom to make sure there is a number in "In[#]" next to the last cell. If there is not - the run did not complete and there is an error somewhere. See troubleshooting section at the end of this notebook for steps.

## Things to Edit: 

In [2]:
DDMMYY = '11JUNE2021'
animals = (1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12)
hpd = [3, 24, 48, 72]


print("HPD [(index #, value)]: "+str(list(enumerate(hpd))), '\n')
print("Done")

HPD [(index #, value)]: [(0, 3), (1, 24), (2, 48), (3, 72)] 

Done


For if/elif loops in hpd_sort( ) function, define number ranges according to the animals in each Hours Post Denervation condition. "elif" loops can be copy-pasted for each number in hpd. Then;

1. Check that animal number ranges are defined correctly, keeping "a" as the variable. 

2. Run the cell above to check that indexes are correct in each if/elif, beginning with 0. (e.g., "hpd[1]" corresponds to the _second_ hour value.)

3. If using Plotly or Seaborn, move on to editing the next cell.

In [3]:
# Defining function to sort hours_post_denervs for appended CSVs
def hpd_sort():
    if 1 <= a <= 3:
        hours_post_denerv = hpd[0]
        excel_outfile_statsheet.write_number(statsRowCounter, 3, hours_post_denerv, innervation_format)
        if innervated is True:
            post_denervs.append(hours_post_denerv)
    
    elif 4 <= a <= 6:
        hours_post_denerv = hpd[1]
        excel_outfile_statsheet.write_number(statsRowCounter, 3, hours_post_denerv, innervation_format)
        if innervated is True:
            post_denervs.append(hours_post_denerv)
    
    elif 7 <= a <= 9:
        hours_post_denerv = hpd[2]
        excel_outfile_statsheet.write_number(statsRowCounter, 3, hours_post_denerv, innervation_format)
        if innervated is True:
            post_denervs.append(hours_post_denerv)
    
    elif 10 <= a <= 12:
        hours_post_denerv = hpd[3]
        excel_outfile_statsheet.write_number(statsRowCounter, 3, hours_post_denerv, innervation_format)
        if innervated is True:
            post_denervs.append(hours_post_denerv)
    
    append_CSVs(filename, animal_number, innervation, hours_post_denerv)

print("Done")

Done


Similarly to hpd_sort( ), define number ranges according to what animals were in each innervation condition. Indexes should also align with those defined above, i.e., "keys[0]" is the same value as "hpd[0]". If there are animals with uninjured tissue data, "uninjured_comparisons.append(df)" can be moved/copy-pasted accordingly, otherwise add # in front. If/elif loops can be added or deleted as necessary.

In [4]:
# Defining function to sort violin files
# The 'keys' list is created later, but does not need to be edited manually.

def violin_df_sort(df):
    if 1 <= int(a) <= 3:
        hpd_dfs[keys[0]].append(df)
        uninjured_comparisons.append(df)
    
    elif 4 <= int(a) <= 6:
        hpd_dfs[keys[1]].append(df)
    
    elif 7 <= int(a) <= 9:
        hpd_dfs[keys[2]].append(df)
    
    elif 10 <= int(a) <= 12:
        hpd_dfs[keys[3]].append(df)

print("Done")

Done


### End Editing

## Packages and objects for all runs

In [5]:
# These packages and modules are necessary for every run
# If using this notebook for the first time on a new computer, 
# new packages may have to be installed.

import pandas as pd
import xlsxwriter
import os
from pathlib import Path
from csv import writer, reader

# Relative file path of working directory
file_path = './'

# Defining variable to contain data .csv files from file_path
all_files = Path(file_path).glob('*_'+DDMMYY+'.csv')

# Creating .xlsx output file; exports to current folder
excel_outfile = xlsxwriter.Workbook('Statistics_'+DDMMYY+'.xlsx')

print("Done")

Done


# Running Statistics

### Creating and formatting _Statistics_ worksheet

In [6]:
# Creating the worksheet and setting it as the first sheet in workbook
try:
    excel_outfile_statsheet = excel_outfile.add_worksheet("Statistics")
except (xlsxwriter.exceptions.DuplicateWorksheetName):
    pass

excel_outfile_statsheet.set_first_sheet()

# Writing and formatting header of Statistics sheet
# Initializing list to contain headers for stats columns
headerList = ['Animal', 'Innervation', 'Hours Post Denerv', 'ROI Count', 'Mean Intensity', 'Median Intensity', 
              'Std Dev', 'Shapiro TestStat', 'Shapiro PValue']

# Initializing two cell formats for general and merged cells
general_format = excel_outfile.add_format({'bold': True, 'align': 'center', 'border': 1})
merge_format = excel_outfile.add_format({'bold': True, 'align': 'center', 'border': 1, 'valign': 'center'})
innervation_format = excel_outfile.add_format({'align': 'center', 'border': 1})
count_format = excel_outfile.add_format({'align': 'center', 'border': 1, 'num_format': 1})
stat_format = excel_outfile.add_format({'align': 'center', 'border': 1, 'num_format': 2})

# Initializing a counter to add headers in sequential columns 
headerColumnCounter = 1

# Looping through column headers and writing to cells in Statistics_DDMMYY.xlsx outfile
for header in headerList:
    general_format.set_bottom(2)
    excel_outfile_statsheet.set_column('B:J', len(str(header)))
    excel_outfile_statsheet.write(1, headerColumnCounter, header, general_format)
    headerColumnCounter += 1

# Merging cells above Shapiro-Wilk values
excel_outfile_statsheet.merge_range('I1:J1', "Shapiro-Wilk", merge_format)

print("Done")

Done


#### p-value significance formatting styles

In [7]:
# Formatting for indicating p value significance
p_signif = excel_outfile.add_format({'align': 'center', 'border': 1, 'num_format': 11, 'bg_color': '#D9FFB3'})
p_insignif = excel_outfile.add_format({'align': 'center', 'border': 1, 'num_format': 11, 'bg_color': '#F08080'})

print("Done")

Done


### Defining Histogram and Probability Plot folders and functions

This cell must be run before statistics cell even if Histograms and Probability Plots will not be exported.

In [8]:
# Importing necessary packages
import pylab
from matplotlib import pyplot

# Creating folder to contain Histogram and ProbPlot .png files
histogram_probplot_directory = "Histograms_and_Probplots"
StatsCharts_ImageFile_path = os.path.join(file_path, histogram_probplot_directory)
try:
    os.mkdir(StatsCharts_ImageFile_path)
except (FileExistsError):
    pass

# Defining function to make histograms
def histogram_maker():
    df = pd.read_csv(csvfile).dropna(axis = 0)
    histo_gram = pyplot.hist(df["Mean"], zorder=3)
    
    title_name = os.path.splitext(csvfile)
    pyplot.title(title_name[0], fontdict={'fontweight': 'bold'})
    pyplot.xlabel('Mean Intensity')
    pyplot.ylabel('Number of Cells')
    pyplot.grid(True, zorder=0)
    pyplot.tight_layout()
    
    histogram_outname = (str(title_name[0]) + "_histogram")
    pyplot.savefig(os.path.join(StatsCharts_ImageFile_path, histogram_outname), dpi=300)
    
    pyplot.close()

# Defining function to make Probability Plots
def probplot_maker():
    df = pd.read_csv(csvfile).dropna(axis = 0)
    prob_plot = stats.probplot(df["Mean"], dist="norm", plot=pylab)
    
    title_name = os.path.splitext(csvfile)
    pylab.title(title_name[0], fontdict={'fontweight': 'bold'})
    pylab.ylabel('Mean Intensity Ordered Values')
    pylab.grid(False)
    pylab.tight_layout()
    
    probplot_outname = (str(title_name[0]) + "_probplot")
    pylab.savefig(os.path.join(StatsCharts_ImageFile_path, probplot_outname), dpi=300)
    
    pylab.close()

print("Done")

Done


### Creating appended CSVs for later Violin plotting

This cell must be run before statistics even if Violins will not be created.

In [9]:
# Creating folder to contain files with appended CSVs for later Violin plots
appended_csv_directory = "Appended_CSVs"
appended_csv_path = os.path.join(file_path, appended_csv_directory)
try:
    os.mkdir(appended_csv_path)
except (FileExistsError):
    pass

# Initializing array of headers for Appended .csv outfiles
appended_headers = ['ROI', 'Area', 'Mean_Intensity', 'Min', 'Max', 'IntDen', 
                    'RawIntDen', 'Animal', 'Innervation', 'Hours_Post_Denerv']

# Defining function to make Appended CSVs
def append_CSVs(filename, animal_number, innervation, hours_post_denerv):
    csv_split = os.path.splitext(filename)
    outcsv_name = str(csv_split[0] + "_Appended.csv")
    csv_toDirectory = os.path.join(appended_csv_path, outcsv_name)
    
    # Opening outfile with defined name 
    with open(csv_toDirectory, 'w', newline='') as outfile:
        csv_writer = writer(outfile, delimiter=',')
        csv_writer.writerow(appended_headers)
        
        # Using current file as infile
        with open(csvfile, 'r') as infile:
            csv_reader = reader(infile)
            next(csv_reader, None)
            
            # Writing row-by-row to outfile
            for row in csv_reader:
                row.append(animal_number)
                row.append(innervation)
                row.append(hours_post_denerv)
                csv_writer.writerow(row)

print("Done")

Done


## Calculating statistics

In [10]:
# Importing necessary package
from scipy import stats

# Defining function to run statistics
def stats_calc(csvfile):
    with open(csvfile, 'r') as infile:
        df = pd.read_csv(infile, usecols=['Mean']).dropna(axis=0)
        
        # Counting # of rows in .csv, corresponds to ROI#
        ROI_count = len(df.axes[0])
        
        # Calculating mean, median, std. deviation
        mean = df.mean()
        median = df.median()
        stddev = df.std()
        
        # Shapiro-Wilk tests
        shapiro_stat, shapiro_p = stats.shapiro(df["Mean"])
        
        return(int(ROI_count), float("%.2f"%mean), float("%.2f"%median), float("%.2f"%stddev),
               float("%.2f"%shapiro_stat), float("%.2e"%shapiro_p))

print("Done")

Done


#### Defining function to write statistics to Statistics_DDMMYY.xlsx outfile

In [11]:
# Loops through stats_calc results and writes to sequential columns
def stats_writer(result):
    excel_outfile_statsheet.write_number(statsRowCounter, 1, int(a), general_format)
    excel_outfile_statsheet.write_string(statsRowCounter, 2, innervation, innervation_format)
    # The column '3' write-out is missing on purpose here
    excel_outfile_statsheet.write_number(statsRowCounter, 4, result[0], count_format)
    excel_outfile_statsheet.write_number(statsRowCounter, 5, result[1], stat_format)
    excel_outfile_statsheet.write_number(statsRowCounter, 6, result[2], stat_format)
    excel_outfile_statsheet.write_number(statsRowCounter, 7, result[3], stat_format)
    excel_outfile_statsheet.write_number(statsRowCounter, 8, result[4], stat_format)
    
    # Formatting p-value output based on significance
    if result[5] < 0.05:
        excel_outfile_statsheet.write_number(statsRowCounter, 9, result[5], p_insignif)
    else:
        excel_outfile_statsheet.write_number(statsRowCounter, 9, result[5], p_signif)

print("Done")

Done


## Main Statistics cell

In [12]:
# Initializing list to contain animal numbers for later ratio table
# Maintains sort (10, 11, ..., n, 1, 2, ...) rather than (1, 2, 3, ..., n)
animal_numbers = list()

# Initializing list to contain values for later writing to Ratios, Mann-Whitney table
# Maintains sorting in order (10, 11, ..., n, 1, 2,...) rather than (1, 2, 3,..., n)
post_denervs = list()

# Initializing lists to contain dataframes and means for later Mann-Whitney tests
# and ratio table population
MW_Is=[]
I_means=list()

MW_Ds=[]
D_means=list()

# Adjusting cell bottom border 
general_format.set_bottom(1)

# Initializing row counter to add data row by row starting after header row
statsRowCounter = 2

# Loops through all #_Condition_DDMMYY.csv files in file_path
for csvfile in sorted(all_files):
    
    filename = csvfile.name
    
    # Creating dataframe from file and dropping NaNs
    df = pd.read_csv(csvfile).dropna(axis=0)
    
    # Creating ProbPlots and saving to Histogram&ProbPlot folder
    probplot_maker()
    
    # Creating Histograms and saving to Histogram&ProbPlot folder
    histogram_maker()
    
    for a in animals:
        # defining for later appending to output .csv
        animal_number = a
        
        # Selecting only Innervated files
        if filename.startswith(str(a) +'_Innervated_'+DDMMYY):
            animal_numbers.append(int(a))
            innervation = 'Innervated'
            innervated = True
            denervated = False
            
            # Appending file to Mann-Whitney dataframes for later test
            MWU_I = pd.read_csv(filename)
            MW_Is.append(MWU_I)
            
            # Calculating stats
            results = [stats_calc(csvfile) for filename in range(a)]
            
            # Looping through innervated files by Hours Post Denervation and writing to appropriate cells
            hpd_sort()

            # Looping through defined stats lists and writing to appropriate cells
            for result in results:
                stats_writer(result)
            
            # Adding calculated mean to I_means list
            if innervated is True:    
                I_means.append(result[1])
            
            statsRowCounter += 1

        # Repeating pattern for Denervated files
        elif filename.startswith(str(a)+'_Denervated_'+DDMMYY):
            innervation = 'Denervated'
            innervated = False
            denervated = True
            
            # Adding files to Mann-Whitney dataframes for later test
            MWU_D = pd.read_csv(filename)
            MW_Ds.append(MWU_D)
            
            results = [stats_calc(csvfile) for filename in range(a)]
            
            for result in results:
                stats_writer(result)
            
            if denervated is True:    
                D_means.append(result[1])
            
            hpd_sort()
            
            statsRowCounter += 1
            
        # Repeating pattern for Uninjured files
        # Does not create Mann-Whitney dataframes
        elif filename.startswith(str(a)+'_Uninjured_'+DDMMYY):
            innervation = 'Uninjured'
            innervated = False
            denervated = False
            
            results = [stats_calc(csvfile) for filename in range(a)]
            
            for result in results:
                stats_writer(result)
            
            hpd_sort()
            
            statsRowCounter += 1
        
        # Repeating pattern for Control files
        # Does not create Mann-Whitney dataframes
        elif filename.startswith(str(a)+'_Control_'+DDMMYY):
            innervation = 'Control'
            innervated = False
            denervated = False
            
            results = [stats_calc(csvfile) for filename in range(a)]
            
            for result in results:
                stats_writer(result)
            
            hpd_sort()
            
            statsRowCounter += 1

print("Done")

Done


### Creating and populating ratio and Mann-Whitney table

#### Formatting table headers

In [13]:
# Merging cells for Mann-Whitney header
excel_outfile_statsheet.merge_range(statsRowCounter+1, 6, statsRowCounter+1, 7, "Mann Whitney", merge_format)

# Creating counters for row and column progression
# ratioRowCounter positions the ratio table 2 cells below main stats table
ratioRowCounter = int(statsRowCounter) + 2
ratioColumnCounter = 3

# Initializing list of ratio table headers
ratioHeaders = ['Animal', 'Hours Post Denerv', 'Ratio', 'MW U', 'MW Pvalue']

# Writing each header to ratio table
for ratioHeader in ratioHeaders:
    excel_outfile_statsheet.write(ratioRowCounter, ratioColumnCounter, ratioHeader, general_format)
    ratioColumnCounter += 1
    
print("Done")

Done


### Calculating ratios to populate table

In order for this cell to function, first run the main statistics cell to calculate means.

In [14]:
# Initializing list to contain calculated ratios
ratios = list()

# Looping through previously calculated means and pairing Innerv to Denerv by animal
# Calculating I:D Mean Intensity ratio and adding to ratios list
for I_mean, D_mean in list(zip(I_means, D_means)):
    mean_ratio = int(I_mean) / int(D_mean)
    ratios.append(mean_ratio)
    
print("Done")

Done


### Populating ratio table with ratios and Mann-Whitney results

The main statistics cells and ratio calculating cell have to be run before this one.

In [15]:
# Pairing data and M-W dataframes to appropriate animal
# Running Mann-Whitney tests by animal
# Writing data to formatted Ratio table

# Creating conditional formatting for ratios
ratio_good = excel_outfile.add_format({'align': 'center', 'border': 1, 'num_format': 2, 'bg_color': '#D9FFB3'})
ratio_bad = excel_outfile.add_format({'align': 'center', 'border': 1, 'num_format': 2, 'bg_color': '#F08080'})

# Updating ratioRowCounter to begin adding data after header row
ratioRowCounter += 1

# Updating ratioColumnCounter to begin adding data in column 3
ratioColumnCounter = 3

# Looping through stats to write to formatted table in .xlsx outfile
for animal, hour, ratio, MW_I, MW_D in list(zip(animal_numbers, post_denervs, ratios, MW_Is, MW_Ds)):
    # Running Mann-Whitney test by Innervated:Denervated data
    MW_U, MW_pvalue = stats.mannwhitneyu(MW_I['Mean'], MW_D['Mean'], alternative="greater")
    general_format.set_bottom(1)
    
    # Writing animals and hours post denervation to table
    excel_outfile_statsheet.write(ratioRowCounter, ratioColumnCounter, int(animal), general_format)
    excel_outfile_statsheet.write(ratioRowCounter, ratioColumnCounter + 1, int(hour), innervation_format)
    
    # Writing ratios and formatting based on value
    if ratio > 1.00:
        excel_outfile_statsheet.write(ratioRowCounter, ratioColumnCounter + 2, ratio, ratio_good)
    else:
        excel_outfile_statsheet.write(ratioRowCounter, ratioColumnCounter + 2, ratio, ratio_bad)
    
    # Writing Mann-Whitney U value
    excel_outfile_statsheet.write(ratioRowCounter, ratioColumnCounter + 3, MW_U, stat_format)
    
    # Writing Mann-Whitney p value and formatting based on significance
    if MW_pvalue <= 0.05:
        excel_outfile_statsheet.write(ratioRowCounter, ratioColumnCounter + 4, MW_pvalue, p_signif)
    else:
        excel_outfile_statsheet.write(ratioRowCounter, ratioColumnCounter + 4, MW_pvalue, p_insignif)
    
    ratioRowCounter += 1
    
print("Done")

Done


# Ratio Scatterplot

This cell can be deleted, no scatterplot will be created.

In [16]:
# Creating new worksheet to contain scatterplot
try:
    ratio_plotsheet = excel_outfile.add_chartsheet('Ratio Scatterplot')
except (xlsxwriter.exceptions.DuplicateWorksheetName):
    pass

# Initializing scatterplot to populate worksheet
ratio_scatterplot = excel_outfile.add_chart({'type': 'scatter'})

# Formatting scatterplot and populating with ratios from Statistics worksheet
ratio_scatterplot.set_title({'name': 'Innervated:Denervated Mean Intensity Ratios', 
                             'name_font': {'name': 'Arial', 'size': 16, 'bold': True}
                            })
ratio_scatterplot.set_x_axis({'name': 'Hours Post Denervation', 
                              'name_font': {'name': 'Arial', 'size': 12, 'bold': True}
                             })
ratio_scatterplot.set_y_axis({'name': 'Ratio with Std. Error', 'name_font': {'name': 'Arial',
                                                            'size': 12, 'bold': True}
                             })
ratio_scatterplot.add_series({'categories': ['Statistics', int(statsRowCounter) + 3, 4, 
                                             int(statsRowCounter)+ 3 + len(animals), 4], 
                              'values': ['Statistics', int(statsRowCounter) + 3, 5, 
                                         int(statsRowCounter) + 3 + len(animals), 5], 'name': 'Ratio', 
                              'y_error_bars': {'type': 'standard_error'}, 
                              'data_labels': {'value': True, 'position': 'right'},
                              'trendline': {'type': 'polynomial', 'order': 3}
                             })
ratio_scatterplot.set_legend({'none': True})

# Committing scatterplot to worksheet
ratio_plotsheet.set_chart(ratio_scatterplot)

print("Done")

Done


# Histograms and Probability Plots

### Sorting plot files

This cell can be deleted if not using Histograms or Probability Plots.

In [17]:
# Initializing lists to contain Histogram and ProbPlot .png files
I_probplot_files = list()
I_histogram_files = list()

D_probplot_files = list()
D_histogram_files = list()

U_probplot_files = list()
U_histogram_files = list()

C_probplot_files = list()
C_histogram_files = list()

# Appending image files to appropriate list
for imagefile in Path(StatsCharts_ImageFile_path).iterdir():
    imagefile_name = imagefile.name
    
    if imagefile_name.endswith('Innervated_'+DDMMYY+'_probplot.png'):
        I_probplot_files.append(imagefile)
    elif imagefile_name.endswith('Innervated_'+DDMMYY+'_histogram.png'): 
        I_histogram_files.append(imagefile)
    elif imagefile_name.endswith('Denervated_'+DDMMYY+'_probplot.png'):
        D_probplot_files.append(imagefile)
    elif imagefile_name.endswith('Denervated_'+DDMMYY+'_histogram.png'): 
        D_histogram_files.append(imagefile)
    elif imagefile_name.endswith('Uninjured_'+DDMMYY+'_probplot.png'):
        U_probplot_files.append(imagefile)
    elif imagefile_name.endswith('Uninjured_'+DDMMYY+'_histogram.png'): 
        U_histogram_files.append(imagefile)
    elif imagefile_name.endswith('Control_'+DDMMYY+'_probplot.png'):
        C_probplot_files.append(imagefile)
    elif imagefile_name.endswith('Control_'+DDMMYY+'_histogram.png'): 
        C_histogram_files.append(imagefile)
        
print("Done")

Done


### Adding plots to outfile worksheet

This cell can be deleted if not using Histograms or Probability Plots. 

In [18]:
# Creating sheet to contain Histograms + ProbPlots
excel_outfile_chartsheet = excel_outfile.add_worksheet("Histograms_ProbPlots")
excel_outfile_chartsheet.hide_gridlines(2)

# Initializing lists for image adding
row_count = 0
column_count = 0

# Adding Innervated files to first two columns of images
for I_probplot_file, I_histogram_file in zip(sorted(I_probplot_files), sorted(I_histogram_files)):
    excel_outfile_chartsheet.insert_image(row_count, column_count, I_probplot_file)
    column_count += 9
    
    excel_outfile_chartsheet.insert_image(row_count, column_count, I_histogram_file)
    row_count += 19
    column_count -= 9

# Resetting to top row but moving right to begin new 2 image columns
row_count = 0
column_count += 18

# Repeating pattern over Denervated files
for D_probplot_file, D_histogram_file in zip(sorted(D_probplot_files), sorted(D_histogram_files)):
    excel_outfile_chartsheet.insert_image(row_count, column_count, D_probplot_file)
    column_count += 9
    
    excel_outfile_chartsheet.insert_image(row_count, column_count, D_histogram_file)
    row_count += 19
    column_count -= 9

row_count = 0
column_count += 18

# Repeating pattern over Uninjured files
for U_probplot_file, U_histogram_file in zip(sorted(U_probplot_files), sorted(U_histogram_files)):
    excel_outfile_chartsheet.insert_image(row_count, column_count, U_probplot_file)
    column_count += 9
    
    excel_outfile_chartsheet.insert_image(row_count, column_count, U_histogram_file)
    row_count += 19
    column_count -= 9

row_count = 0
column_count += 18

# Repeating pattern over Neg. Ctrl files
for C_probplot_file, C_histogram_file in zip(sorted(C_probplot_files), sorted(C_histogram_files)):
    excel_outfile_chartsheet.insert_image(row_count, column_count, C_probplot_file)
    column_count += 9
    
    excel_outfile_chartsheet.insert_image(row_count, column_count, C_histogram_file)
    row_count += 19
    column_count -= 9

print("Done")

Done


# Violin Plots

All of these cells can be deleted if you don't want Violin plots. 

### Initializing dataframes for plotting

Run this even if only using one violin plotting program.

In [19]:
# Initializing empty dataframes to contain data as necessary for violins

all_traces= []

all_innervated= []
all_denervated= []
all_uninjured= []
all_control= []

uninjured_comparisons= []
innervation_split= []

# Initializing file lists for Plotly and Seaborn parsing
appended_files = list()
plotly_files= list()
seaborn_files= list()

# Adding .csvs to each list for later plotting
for appended_csv in Path(appended_csv_path).iterdir():
    plotly_files.append(appended_csv)
    seaborn_files.append(appended_csv)
    appended_files.append(appended_csv)
    
print("Done")

Done


### Sorting files to appropriate dataframes

Run the next **three** cells even if only using one plotting program.

In [20]:
# Initializing dictionaries to contain dataframes by hours post denervation
# Will concatenate dataframes independent of actual values in hpd

hpd_dfs={}
hpd_concats={}

# Initializing separate lists from hpd to use later, because
#   hpd is consumed in the next for loop.
plotly_hours=list()
seaborn_hours=list()

# Setting each hpd as hpd_dfs key
for h in hpd:
    hpd_dfs[h] = []
    plotly_hours.append(h)
    seaborn_hours.append(h)

# Defining keys to use in later functions
keys=list(hpd_dfs.keys())

print('Done')

Done


In [21]:
# Appends to appropriate data frames for later plotting
for appended_file in sorted(appended_files):
    csv_name = appended_file.name
    df = pd.read_csv(appended_file)
    for a in animals:
        if csv_name.startswith(str(a) + '_Innervated_'+DDMMYY+'_Appended'):
            all_innervated.append(df)
            all_traces.append(df)
            innervation_split.append(df)
            violin_df_sort(df)
            
        elif csv_name.startswith(str(a) + '_Denervated_'+DDMMYY+'_Appended'):
            all_denervated.append(df)
            all_traces.append(df)
            innervation_split.append(df)
            violin_df_sort(df)

        elif csv_name.startswith(str(a) + '_Uninjured_'+DDMMYY+'_Appended'):
            all_uninjured.append(df)
            all_traces.append(df)
            violin_df_sort(df)
            
        elif csv_name.startswith(str(a) + '_Control_'+DDMMYY+'_Appended'):
            all_control.append(df)
            all_traces.append(df)
            violin_df_sort(df)

# Concatenating dataframes sorted above
try:
    all_traces = pd.concat(all_traces, ignore_index=True)
except (ValueError):
    pass

try:
    all_innervated = pd.concat(all_innervated, ignore_index=True)
except (ValueError):
    pass

try:
    all_denervated = pd.concat(all_denervated, ignore_index=True)
except (ValueError):
    pass

try:
    all_uninjured = pd.concat(all_uninjured, ignore_index=True)
    uninjured_exist = True
except (ValueError):
    uninjured_exist=False
    pass

try:
    all_control=pd.concat(all_control, ignore_index=True)
    control_exist=True
except (ValueError):
    control_exist=False
    pass

try:
    uninjured_comparison = pd.concat(uninjured_comparisons, ignore_index=True)
    uninjureds_exist=True
except (ValueError):
    uninjureds_exist=False
    pass

try:
    innervation_split = pd.concat(innervation_split, ignore_index=True)
except:
    pass

print("Done")

Done


In [22]:
# Concatenating dataframes based on hpd
# Appending [ key : concatenated dataframe] to hpd_concats dict.
for key, values in hpd_dfs.items():
    df_out = pd.concat(values, ignore_index=True)
    hpd_concats[key] = []
    hpd_concats[key].append(df_out)

print('Done')

Done


## Plotly Violins

All of these cells can be deleted if you don't want any Plotly violins.
Individual cells can be deleted depending on what violins you do want.

In [24]:
# Importing necessary packages
import plotly.graph_objects as go
import plotly.express as px

# Initializing directories to contain Plotly static violins
plotly_violin_directory = "Plotly_PNGs"
plotly_path = os.path.join(file_path, plotly_violin_directory)
try:
    os.mkdir(plotly_path)
except (FileExistsError):
    pass

# Initializing directories to contain Plotly interactive violins
plotly_html_directory = "Plotly_HTMLs"
html_path = os.path.join(file_path, plotly_html_directory)
try:
    os.mkdir(html_path)
except (FileExistsError):
    pass

print("Done")

Done


#### Violins by hours post denerv condition

This cell can be deleted.

In [26]:
# Defining the function to make each violin
def hpd_plotly_violin(key, df):
    comparison = go.Figure()
    comparison.add_trace(go.Violin(x=df['Animal'][ df['Innervation'] == 'Innervated' ],
                        y=df['Mean_Intensity'][ df['Innervation'] == 'Innervated' ],
                        legendgroup='Innervated', scalegroup='Innervated', name='In',
                        side='negative',
                        line_color='rgb(136, 204, 238)')
             )
    comparison.add_trace(go.Violin(x=df['Animal'][ df['Innervation'] == 'Denervated' ],
                        y=df['Mean_Intensity'][ df['Innervation'] == 'Denervated' ],
                        legendgroup='Denervated', scalegroup='Denervated', name='De',
                        side='positive',
                        line_color='rgb(221, 204, 119)')
             )
    comparison.update_traces(box_visible=True, meanline_visible=True, 
                              points='outliers')
    comparison.update_yaxes(title_text='Mean Intensity')
    comparison.update_xaxes(title_text='Animal')
    comparison.update_layout(title_text=str(key)+' Hours Post Denerv',
                                     violingap=0, violinmode='overlay', yaxis_range=[-5000, 65000])
    comparison.write_image('Plotly_PNGs/'+str(key)+'hpd_Animals.png')
    comparison.write_html('Plotly_HTMLs/'+str(key)+'hpd_Animals.html')

# Looping through dictionary of concatenated dataframes and creating violin
for key, values in hpd_concats.items():
    df = values[0]
    hpd_plotly_violin(key, df)

print('Done')

Done


#### Individual violin plots

This cell can be deleted.

In [27]:
# Defining function to create individual Plotly static and interactive violins
def individual_plotly_violin(df, innervation):
    violin1 = px.violin(df, y="Mean_Intensity", x="Animal", box=True, points='all')
    violin1.update_layout(title_text=str(a)+ " " + innervation, yaxis_range=[-5000, 65000])
    violin1.update_traces(meanline_visible=True)
    violin1.write_image("Plotly_PNGs/" + str(a)+ "_" + innervation + ".png")
    violin1.write_html("Plotly_HTMLs/" + str(a)+ "_" + innervation + ".html")

# Creating individual Plotly static and interactive violins
for plotly_file in sorted(plotly_files):
    csv_name = plotly_file.name
    df = pd.read_csv(plotly_file)
    for a in animals:
        if csv_name.startswith(str(a) + '_Innervated_'+DDMMYY+'_Appended'):            
            innervation = 'Innervated'
            individual_plotly_violin(df, innervation)
            
        elif csv_name.startswith(str(a) + '_Denervated_'+DDMMYY+'_Appended'):
            innervation = "Denervated"
            individual_plotly_violin(df, innervation)

        elif csv_name.startswith(str(a) + '_Uninjured_'+DDMMYY+'_Appended'):
            innervation = "Uninjured"
            individual_plotly_violin(df, innervation)
            
        elif csv_name.startswith(str(a) + '_Control_'+DDMMYY+'_Appended'):
            innervation = "Control"
            individual_plotly_violin(df, innervation)

print("Done")

Done


#### Violins by Innervation

This cell can be deleted.

In [28]:
# Creating Plotly violins for each Innervation by Animal

# Innervated
try:
    plotly_innervated = px.violin(all_innervated, y="Mean_Intensity", x="Animal", 
                              color="Animal", box=True, points="outliers",
                              hover_data=all_innervated.columns,
                              color_discrete_sequence=px.colors.qualitative.Safe
                             )

    plotly_innervated.update_layout(title_text="All Innervated", yaxis_range=[-5000, 65000])
    plotly_innervated.update_traces(width=0.75, meanline_visible=True)
    plotly_innervated.update_yaxes(title_text='Mean Intensity')
    plotly_innervated.update_xaxes(title_text='Animal', tick0=1, dtick=1)
    plotly_innervated.write_image("Plotly_PNGs/All_Innervated.png")
    plotly_innervated.write_html("Plotly_HTMLs/All_Innervated.html")

except (AttributeError) or (ValueError):
    pass

# Denervated
try:
    plotly_denervated = px.violin(all_denervated, y="Mean_Intensity", x="Animal", 
                              color="Animal", box=True, points="outliers",
                              hover_data=all_denervated.columns,
                             color_discrete_sequence=px.colors.qualitative.Safe)

    plotly_denervated.update_layout(title_text="All Denervated", yaxis_range=[-5000, 65000])
    plotly_denervated.update_traces(width=0.75, meanline_visible=True)
    plotly_denervated.update_yaxes(title_text='Mean Intensity')
    plotly_denervated.update_xaxes(title_text='Animal', tick0=1, dtick=1)
    plotly_denervated.write_image("Plotly_PNGs/All_Denervated.png")
    plotly_denervated.write_html("Plotly_HTMLs/All_Denervated.html")

except (AttributeError) or (ValueError):
    pass

# Uninjured
try:
    plotly_uninjured = px.violin(all_uninjured, y="Mean_Intensity", x="Animal", 
                              color="Animal", box=True, points="outliers",
                              hover_data=all_uninjured.columns,
                            color_discrete_sequence=px.colors.qualitative.Safe)

    plotly_uninjured.update_layout(title_text="All Uninjured", yaxis_range=[-5000, 65000])
    plotly_uninjured.update_traces(width=0.75, meanline_visible=True)
    plotly_uninjured.update_yaxes(title_text='Mean Intensity')
    plotly_uninjured.write_image("Plotly_PNGs/All_Uninjured.png")
    plotly_uninjured.write_html("Plotly_HTMLs/All_Uninjured.html")

except (AttributeError) or (ValueError):
    pass

# Control
try:
    plotly_control = px.violin(all_control, y="Mean_Intensity", x="Animal", 
                              color="Animal", box=True, points="all",
                              hover_data=all_control.columns,
                          color_discrete_sequence=px.colors.qualitative.Safe)

    plotly_control.update_layout(title_text="Negative Control", yaxis_range=[-5000, 65000])
    plotly_control.update_traces(width=0.75, meanline_visible=True)
    plotly_control.update_yaxes(title_text='Mean Intensity')
    plotly_control.write_image("Plotly_PNGs/Negative_CTRLs.png")
    plotly_control.write_html("Plotly_HTMLs/Negative_CTRLs.html")

except (AttributeError) or (ValueError):
    pass
    
print("Done")

Done


#### Violin for all data

This cell can be deleted.

In [29]:
# Creating Plotly violin to display all data

all_data = go.Figure()

try:
    all_data.add_trace(go.Violin(x=all_innervated['Animal'][ all_innervated['Innervation'] == 'Innervated' ],
                        y=all_innervated['Mean_Intensity'][ all_innervated['Innervation'] == 'Innervated' ],
                        legendgroup='Innervated', scalegroup='Innervated', name='In',
                        line_color='rgb(136, 204, 238)')
             )

except (AttributeError) or (ValueError) or (TypeError):
    pass

try:
    all_data.add_trace(go.Violin(x=all_denervated['Animal'][ all_denervated['Innervation'] == 'Denervated' ],
                        y=all_denervated['Mean_Intensity'][ all_denervated['Innervation'] == 'Denervated' ],
                        legendgroup='Denervated', scalegroup='Denervated', name='De',
                        line_color='rgb(221, 204, 119)')
             )
except (AttributeError) or (ValueError) or (TypeError):
    pass

if uninjured_exist is True:
    all_data.add_trace(go.Violin(x=all_uninjured['Animal'][ all_uninjured['Innervation'] == 'Uninjured' ],
                        y=all_uninjured['Mean_Intensity'][ all_uninjured['Innervation'] == 'Uninjured' ],
                        legendgroup='Uninjured', scalegroup='Uninjured', name='Un',
                        line_color='rgb(204, 102, 119)')
             )
else:
    pass

if control_exist is True:
    all_data.add_trace(go.Violin(x=all_control['Animal'][ all_control['Innervation'] == 'Control' ],
                        y=all_control['Mean_Intensity'][ all_control['Innervation'] == 'Control' ],
                        legendgroup='Control', scalegroup='Control', name='Ctrl',
                        line_color='rgb(17, 119, 51)')
             )
else:
    pass

all_data.update_traces(box_visible=True, meanline_visible=True, points=False)
all_data.update_yaxes(title_text='Mean Intensity')
all_data.update_xaxes(title_text='Animal', tick0=1, dtick=1)
all_data.update_layout(title_text="All Data", violinmode='group', yaxis_range=[-5000, 65000])

all_data.write_image("Plotly_PNGs/All_Data.png")
all_data.write_html("Plotly_HTMLs/All_Data.html")

print("Done")

Done


#### Violin for animals with uninjured data

This cell can be deleted.

In [30]:
# Creating Plotly violin to compare limbs of animals with Uninjured data

if uninjureds_exist is True:
    plotly_UninjuredAnimals = go.Figure()
    plotly_UninjuredAnimals.add_trace(go.Violin(x=uninjured_comparison['Animal'][ uninjured_comparison['Innervation'] == 'Innervated' ],
                        y=uninjured_comparison['Mean_Intensity'][ uninjured_comparison['Innervation'] == 'Innervated' ],
                        legendgroup='In', scalegroup='In', name='In',
                        line_color='rgb(136, 204, 238)')
             )
    plotly_UninjuredAnimals.add_trace(go.Violin(x=uninjured_comparison['Animal'][ uninjured_comparison['Innervation'] == 'Denervated' ],
                        y=uninjured_comparison['Mean_Intensity'][ uninjured_comparison['Innervation'] == 'Denervated' ],
                        legendgroup='De', scalegroup='De', name='De',
                        line_color='rgb(221, 204, 119)')
             )
    plotly_UninjuredAnimals.add_trace(go.Violin(x=uninjured_comparison['Animal'][ uninjured_comparison['Innervation'] == 'Uninjured' ],
                        y=uninjured_comparison['Mean_Intensity'][ uninjured_comparison['Innervation'] == 'Uninjured' ],
                        legendgroup='Un', scalegroup='Un', name='Un',
                        line_color='rgb(204, 102, 119)')
             )

    plotly_UninjuredAnimals.update_traces(box_visible=True, meanline_visible=True, points='outliers')
    plotly_UninjuredAnimals.update_yaxes(title_text='Mean Intensity')
    plotly_UninjuredAnimals.update_xaxes(title_text='Animal')
    plotly_UninjuredAnimals.update_layout(title_text="Animals with Uninjured Limbs", violinmode='group',
                                     yaxis_range=[-5000, 65000])

    plotly_UninjuredAnimals.write_image("Plotly_PNGs/Animals_WithUninjured.png")
    plotly_UninjuredAnimals.write_html("Plotly_HTMLs/Animals_WithUninjured.html")

else:
    pass

print("Done")

Done


#### Split violin innervated:denervated

This cell can be deleted.

In [31]:
# Creating Plotly violin to compare Innervated:Denervated by animal

innervated_denervated_split = go.Figure()

innervated_denervated_split.add_trace(go.Violin(x=innervation_split['Animal'][ innervation_split['Innervation'] == 'Innervated' ],
                        y=innervation_split['Mean_Intensity'][ innervation_split['Innervation'] == 'Innervated' ],
                        alignmentgroup='Innervated', legendgroup='Innervated', scalegroup='Innervated', name='In',
                        box_visible=True, meanline_visible=True, points='outliers', side='negative',
                        line_color='rgb(136, 204, 238)')
             )
innervated_denervated_split.add_trace(go.Violin(x=innervation_split['Animal'][ innervation_split['Innervation'] == 'Denervated' ],
                        y=innervation_split['Mean_Intensity'][ innervation_split['Innervation'] == 'Denervated' ],
                        alignmentgroup='Denervated', legendgroup='Denervated', scalegroup='Denervated', name='De',
                        box_visible=True, meanline_visible=True, points='outliers', side='negative',
                        line_color='rgb(221, 204, 119)')
             )

innervated_denervated_split.update_layout(title_text="Innervated vs. Denervated", violingap=0, violinmode='overlay',
                                         yaxis_range=[-5000, 65000])
innervated_denervated_split.update_yaxes(title_text='Mean Intensity')
innervated_denervated_split.update_xaxes(title_text='Animal', tick0=1, dtick=1)

innervated_denervated_split.write_image("Plotly_PNGs/Innerv_VsDenerv.png")
innervated_denervated_split.write_html("Plotly_HTMLs/Innerv_VsDenerv.html")

print("Done")

Done


### Exporting Plotly Violins to outfile

This cell can be deleted.

In [32]:
# Creating and formatting worksheet to contain non-individual violins
try:
    excel_outfile_violinsheet = excel_outfile.add_worksheet("Comparison Plotly Violins")
except (xlsxwriter.exceptions.DuplicateWorksheetName):
    pass
    
excel_outfile_violinsheet.hide_gridlines(2)
excel_outfile_violinsheet.write(0, 0, 'Interactive violin plots can be found in the Plotly_HTMLs folder')

# Creating and formatting worksheet to contain individual violins
try:
    excel_outfile_plotlyviolins = excel_outfile.add_worksheet("Individual Plotly Violins")
except (xlsxwriter.exceptions.DuplicateWorksheetName):
    pass

excel_outfile_plotlyviolins.hide_gridlines(2)
excel_outfile_plotlyviolins.write(0, 0, 'Interactive violin plots can be found in the Plotly_HTMLs folder')

static_plotlys = list()

extraRowCounter = 27
violinColumnCounter = 1
hp_columnCounter = 1

for plotly_violin in Path(plotly_path).iterdir():
    violin_name = plotly_violin.name
    if violin_name.endswith('.png'):
        static_plotlys.append(plotly_violin)

# Adding individual violins to Individual Violins worksheet
for static_file in (sorted(static_plotlys)):
    staticfile_name = static_file.name
    for r in animals:
        extraColumnCounter = 1
        if staticfile_name.startswith(str(r) + '_Innervated'):
            excel_outfile_plotlyviolins.insert_image(extraRowCounter - 26, extraColumnCounter, static_file)
            extraRowCounter += 26
            break
    
        elif staticfile_name.startswith(str(r) + '_Denervated'):
            extraRowCounter -= 26
            extraColumnCounter += 11
            excel_outfile_plotlyviolins.insert_image(extraRowCounter, extraColumnCounter, static_file)
            extraRowCounter += 26
            break
      
        elif staticfile_name.startswith(str(r) + '_Uninjured'):
            extraRowCounter -= 26
            extraColumnCounter += 22
            excel_outfile_plotlyviolins.insert_image(extraRowCounter - 26, extraColumnCounter, static_file)
            extraRowCounter += 26
            break
       
        elif staticfile_name.startswith(str(r) + '_Control'):
            extraRowCounter -= 26
            extraColumnCounter += 33
            excel_outfile_plotlyviolins.insert_image(extraRowCounter, extraColumnCounter, static_file)
            extraRowCounter += 26
            break
    
    # Adding "All_Condition" violins to Comparison Violins worksheet
    if staticfile_name.startswith('All_'):
        excel_outfile_violinsheet.insert_image(1, violinColumnCounter, static_file)
        violinColumnCounter += 11
    
    # Adding Split Innervated:Denervated violin
    if staticfile_name.startswith('Innerv_VsDenerv'):
        excel_outfile_violinsheet.insert_image(27, 1, static_file)

    # Sequentially adding Violins for each HPD condition
    for h in plotly_hours:
        if staticfile_name.startswith(str(h) + 'hpd_Animals'):
            excel_outfile_violinsheet.insert_image(53, hp_columnCounter, static_file)
            hp_columnCounter += 11
    
    # Adding Violin for Animals with Uninjured data to compare
    if staticfile_name.startswith('Animals_WithUninjured'):
        excel_outfile_violinsheet.insert_image(27, 12, static_file)
    
    # Adding Violin for negative controls
    if staticfile_name.startswith('Negative_'):
        excel_outfile_violinsheet.insert_image(79, 1, static_file)

print("Done")

Done


## Seaborn violins

All of these cells can be deleted if you don't want any Seaborn violins.
Individual cells can be deleted depending on what violins you do want.

In [33]:
# Importing seaborn package
import seaborn as sns

# Creating folder to contain Seaborn static violins
seaborn_violin_directory = "Seaborn_Violins"
seaborn_path = os.path.join(file_path, seaborn_violin_directory)
try:
    os.mkdir(seaborn_path)
except (FileExistsError):
    pass

print("Done")

Done


#### Creating seaborn violins to compare innervation by Animal # and hours post denervation

This cell can be deleted.

In [34]:
# Defining function to make violins based on hpd condition
def hpd_seaborn_violin(key, df):
    ax = sns.violinplot(x="Animal", y="Mean_Intensity", hue="Innervation",
                   data=df, palette="colorblind", inner="quartile",
                    scale="count").set(title=str(key)+' Hours Post Denerv', 
                                       ylabel="Mean Intensity", ylim=(-5000, 65000))
    pyplot.tight_layout()
    pyplot.savefig(os.path.join(seaborn_path, str(key)+"hpd_Animals.png"), dpi=300)
    pyplot.close()

# Creating violins for each condition in concatenated dataframe dictionary
for key, values in hpd_concats.items():
    df = values[0]
    hpd_seaborn_violin(key, df)

print("Done")

Done


#### Seaborn plots for individual violins

This cell can be deleted.

In [35]:
# Defining function to make violin plots for each individual distribution
def seaborn_individual(df, innervation):
            sns.set_theme(style='whitegrid')
            ax = sns.violinplot(data=df, x='Animal', y='Mean_Intensity', 
                           inner='box', linewidth=1).set(title=str(s)+ ' '+ innervation, 
                                                              ylabel='Mean Intensity', ylim=(-5000, 65000))
            pyplot.tight_layout()
            pyplot.savefig(os.path.join(seaborn_path, str(s) + "_" + innervation) + '.png', dpi=300)
            pyplot.close()

# Creating individual violins
for seaborn_file in sorted(seaborn_files):
    csv_name = seaborn_file.name
    df = pd.read_csv(seaborn_file)
    for s in animals:
        if csv_name.startswith(str(s) + '_Innervated_'+DDMMYY+'_Appended'):
            innervation = "Innervated"
            seaborn_individual(df, innervation)
        
        elif csv_name.startswith(str(s) + '_Denervated_'+DDMMYY+'_Appended'):
            innervation = "Denervated"
            seaborn_individual(df, innervation)
            
        elif csv_name.startswith(str(s) + '_Uninjured_'+DDMMYY+'_Appended'):
            innervation = "Uninjured"
            seaborn_individual(df, innervation)
        
        elif csv_name.startswith(str(s) + '_Control_'+DDMMYY+'_Appended'):
            innervation = "Control"
            seaborn_individual(df, innervation)
            
print("Done")

Done


#### Plots by innervation

This cell can be deleted.

In [36]:
# Creating seaborn figures by innervation

# Innervated
ax = sns.violinplot(x='Animal', y='Mean_Intensity', data=all_innervated, 
                    scale='count', inner="quartile", palette='colorblind').set(title='All Innervated', 
                                                         ylabel='Mean Intensity', ylim=(-5000, 65000))
pyplot.tight_layout()
pyplot.savefig(os.path.join(seaborn_path, 'All_Innervated.png'), dpi=300)
pyplot.close()

# Denervated
ax = sns.violinplot(x='Animal', y='Mean_Intensity', data=all_denervated, 
                    scale='count', inner="quartile", palette='colorblind').set(title='All Denervated', 
                                                         ylabel='Mean Intensity', ylim=(-5000, 65000))
pyplot.tight_layout()
pyplot.savefig(os.path.join(seaborn_path, 'All_Denervated.png'), dpi=300)
pyplot.close()

# Uninjured
if uninjured_exist is True:
    ax = sns.violinplot(x='Animal', y='Mean_Intensity', data=all_uninjured, 
                    scale='count', inner="quartile", palette='colorblind').set(title='All Uninjured', 
                                                         ylabel='Mean Intensity', ylim=(-5000, 65000))
    pyplot.tight_layout()
    pyplot.savefig(os.path.join(seaborn_path, 'All_Uninjured.png'), dpi=300)
    pyplot.close()
    
else:
    pass

# Control
if control_exist is True:
    ax = sns.violinplot(x='Animal', y='Mean_Intensity', data=all_control, 
                    scale='count', inner="quartile", palette='colorblind').set(title='Negative Control', 
                                                         ylabel='Mean Intensity', ylim=(-5000, 65000))
    pyplot.tight_layout()
    pyplot.savefig(os.path.join(seaborn_path, 'Negative_CTRLs.png'), dpi=300)
    pyplot.close()

else:
    pass

print("Done")

Done


#### Violin for all data

This cell can be deleted.

In [37]:
# Creating seaborn violin to compare all data

ax = sns.violinplot(x='Animal', y='Mean_Intensity', hue="Innervation",
                   data=all_traces, scale='count', palette="colorblind", 
                    inner="quartile").set(title='All Data', ylabel='Mean Intensity', ylim=(-5000, 65000))
pyplot.tight_layout()
pyplot.savefig(os.path.join(seaborn_path, 'All_Data.png'), dpi=300)
pyplot.close()

print("Done")

Done


#### Violin for animals with uninjured tissue

This cell can be deleted.

In [38]:
# Creating seaborn violins to compare animals with Uninjured tissue

if uninjureds_exist is True:
    ax = sns.violinplot(x="Animal", y="Mean_Intensity", data=uninjured_comparison, 
                    scale="count", hue="Innervation", palette='colorblind',
                    inner="quartile").set(title='Animals with Uninjured Limbs', 
                                          ylabel='Mean Intensity', ylim=(-5000, 65000))
    pyplot.tight_layout()
    pyplot.savefig(os.path.join(seaborn_path, "Animals_WithUninjured.png"), dpi=300)
    pyplot.close()

else:
    pass

print("Done")

Done


#### Split violin for innervated:denervated

This cell can be deleted. 

In [39]:
# Creating seaborn violin to compare innervated vs. denervated by animal

ax = sns.violinplot(x="Animal", y="Mean_Intensity", hue="Innervation", 
                    data=innervation_split, split=True, palette="colorblind", scale='count', 
                    inner="quartile").set(title='Innervated vs. Denervated', 
                                          ylabel='Mean Intensity', ylim=(-5000, 65000))
pyplot.tight_layout()
pyplot.savefig(os.path.join(seaborn_path, "Innerv_VsDenerv.png"), dpi=300)
pyplot.close()

print("Done")

Done


### Exporting seaborn Violins to Excel outfile

This cell can be deleted.

In [40]:
# Creating worksheet to contain seaborn violins.
try:
    excel_outfile_seabornsheet = excel_outfile.add_worksheet("Comparison Seaborn Violins")
except (xlsxwriter.exceptions.DuplicateWorksheetName):
    pass
    
excel_outfile_seabornsheet.hide_gridlines(2)

try:
    excel_outfile_seabornviolins = excel_outfile.add_worksheet("Individual Seaborn Violins")
except (xlsxwriter.exceptions.DuplicateWorksheetName):
    pass

excel_outfile_seabornviolins.hide_gridlines(2)

seaborn_violins = list()

seabornRowCounter = 21
seabornColumnCounter = 1
extraColumnCounter = 1
hoursColumnCounter = 1

for seaborn_violin in Path(seaborn_path).iterdir():
    seaborn_name = seaborn_violin.name
    if seaborn_name.endswith('.png'):
        seaborn_violins.append(seaborn_violin)

# Adding individual seaborn violins to their respective sheet
for violin in (sorted(seaborn_violins)):
    violin_name = violin.name
    
    for r in animals:
        seabornColumnCounter = 1
        
        if violin_name.startswith(str(r) + '_Innervated'):
            excel_outfile_seabornviolins.insert_image(seabornRowCounter - 20, seabornColumnCounter, violin)
            seabornRowCounter += 20
            break
    
        elif violin_name.startswith(str(r) + '_Denervated'):
            seabornColumnCounter += 9
            seabornRowCounter -= 20
            excel_outfile_seabornviolins.insert_image(seabornRowCounter, seabornColumnCounter, violin)
            seabornRowCounter += 20
            break
      
        elif violin_name.startswith(str(r) + '_Uninjured'):
            seabornColumnCounter += 18
            seabornRowCounter -= 20
            excel_outfile_seabornviolins.insert_image(seabornRowCounter - 20, seabornColumnCounter, violin)
            seabornRowCounter += 20
            break
       
        elif violin_name.startswith(str(r) + '_Control'):
            seabornColumnCounter += 27
            seabornRowCounter -=20
            excel_outfile_seabornviolins.insert_image(seabornRowCounter, seabornColumnCounter, violin)
            seabornRowCounter += 20
            break
    
    # Adding All_Condition violins to Comparison sheet
    if violin_name.startswith('All_'):
        excel_outfile_seabornsheet.insert_image(1, extraColumnCounter, violin)
        extraColumnCounter += 9
    
    # Adding Split Innervated:Denervated violin
    if violin_name.startswith('Innerv_VsDenerv'):
        excel_outfile_seabornsheet.insert_image(21, 1, violin)
    
    # Adding violin for each hpd condition in hpd
    for h in seaborn_hours:
        if violin_name.startswith(str(h)+'hpd_Animals'):
            excel_outfile_seabornsheet.insert_image(41, hoursColumnCounter, violin)
            hoursColumnCounter +=9
    
    # Adding violin for animals with uninjured data
    if violin_name.startswith('Animals_WithUninjured'):
        excel_outfile_seabornsheet.insert_image(21, 10, violin)
     
    # Adding violin for negative control
    if violin_name.startswith('Negative_'):
        excel_outfile_seabornsheet.insert_image(62, 1, violin)

print("Done")

Done


#### Saving and closing Statistics_DDMMYY.xlsx outfile

This command must be run every time.

In [41]:
excel_outfile.close()

print("Done")

Done


# Troubleshooting:

If you received an error, the ratio scatterplot is screwy, or there is data/files missing from the output files or Excel outfile, try these steps before rerunning.

1. Check the file names!!! Make sure the ending is the same across all files. Make sure there aren't any spaces or unintended characters. 

2. Check the ranges defined by hpd_sort( ) and violin_df_srt(df) functions.

3. Check the series range on the scatterplot to make sure it's plotting all values.